## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [2]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv');
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Let's glance at the dataset:

In [5]:
'''
Get the total number of documents
'''
print(len(documents))

300000


In [6]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [12]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
nltk.download('wordnet')
np.random.seed(400)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\US\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [10]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [11]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [48]:
# Functions to perform pre processing on the entire dataset 

def stem_and_lemmatize(text):

    """ Returns a text after performing stemming and lemmatization."""
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    """ Returns tokenized text after catering for stopwords and applying stemming and lemmatization."""
    return [stem_and_lemmatize(word) for word in simple_preprocess(text, min_len=4) if word not in STOPWORDS]

In [49]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [55]:
processed_docs = documents.iloc[:, 0].map(preprocess)

In [56]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [57]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [129]:
'''
We can check if our dictionary is implemented correctly.
'''
i = 0
for k, v in dictionary.items():
    print(f"Id: {k}, Value: {v}")
    if i == 10:
        break
    i += 1

Id: 0, Value: broadcast
Id: 1, Value: communiti
Id: 2, Value: decid
Id: 3, Value: licenc
Id: 4, Value: awar
Id: 5, Value: defam
Id: 6, Value: wit
Id: 7, Value: call
Id: 8, Value: infrastructur
Id: 9, Value: protect
Id: 10, Value: summit


**Gensim filter_extremes**

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [130]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [141]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''

bow_corpus = [dictionary.doc2bow(text) for text in processed_docs]

In [142]:
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [156]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

# Now we check if our BoW model is working correctly
for id, count in bow_doc_4310:
    print(f"Word {id} (\"{dictionary[id]}\") appears {count} time(s).")

Word 71 ("bushfir") appears 1 time(s).
Word 107 ("help") appears 1 time(s).
Word 462 ("rain") appears 1 time(s).
Word 3530 ("dampen") appears 1 time(s).


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [167]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [170]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf = tfidf[bow_corpus]

In [172]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [175]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
from gensim.models import LdaMulticore

lda_model = LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=10)

In [176]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.018*"talk" + 0.016*"deal" + 0.016*"jail" + 0.015*"trial" + 0.013*"appeal" + 0.012*"stand" + 0.011*"trade" + 0.011*"court" + 0.011*"china" + 0.011*"timor"


Topic: 1 
Words: 0.022*"australia" + 0.020*"forc" + 0.018*"open" + 0.016*"iraq" + 0.016*"inquiri" + 0.015*"victim" + 0.014*"begin" + 0.013*"aust" + 0.012*"lead" + 0.012*"australian"


Topic: 2 
Words: 0.072*"polic" + 0.036*"charg" + 0.035*"kill" + 0.031*"face" + 0.023*"attack" + 0.022*"investig" + 0.018*"court" + 0.016*"probe" + 0.015*"drug" + 0.014*"arrest"


Topic: 3 
Words: 0.026*"rise" + 0.023*"death" + 0.020*"case" + 0.019*"return" + 0.018*"high" + 0.017*"market" + 0.016*"price" + 0.015*"hear" + 0.014*"sale" + 0.011*"find"


Topic: 4 
Words: 0.031*"water" + 0.019*"warn" + 0.018*"secur" + 0.016*"test" + 0.015*"concern" + 0.014*"blaze" + 0.012*"prompt" + 0.011*"cost" + 0.011*"fear" + 0.011*"resid"


Topic: 5 
Words: 0.036*"crash" + 0.026*"hospit" + 0.023*"worker" + 0.020*"union" + 0.018*"die" + 0.013*"protest" 

In [177]:
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 10)

In [178]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.017*"miss" + 0.014*"search" + 0.010*"crew" + 0.009*"station" + 0.008*"plant" + 0.008*"grower" + 0.008*"rescu" + 0.008*"hill" + 0.008*"break" + 0.007*"light"


Topic: 1 Word: 0.013*"water" + 0.010*"firefight" + 0.010*"bushfir" + 0.008*"blaze" + 0.007*"control" + 0.007*"condit" + 0.007*"job" + 0.007*"airport" + 0.006*"restrict" + 0.006*"upgrad"


Topic: 2 Word: 0.012*"guilti" + 0.009*"wind" + 0.008*"plead" + 0.007*"tiger" + 0.007*"uranium" + 0.007*"retir" + 0.007*"farm" + 0.007*"cyclon" + 0.006*"recycl" + 0.005*"ralli"


Topic: 3 Word: 0.026*"polic" + 0.021*"charg" + 0.020*"crash" + 0.017*"kill" + 0.015*"investig" + 0.014*"jail" + 0.013*"court" + 0.013*"murder" + 0.011*"death" + 0.010*"woman"


Topic: 4 Word: 0.016*"govt" + 0.011*"plan" + 0.011*"fund" + 0.011*"council" + 0.010*"health" + 0.010*"urg" + 0.008*"indigen" + 0.008*"chang" + 0.008*"group" + 0.007*"union"


Topic: 5 Word: 0.009*"england" + 0.009*"aussi" + 0.009*"open" + 0.009*"final" + 0.008*"world" + 0.008*"vic

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [179]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [180]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4201993942260742	 
Topic: 0.037*"seek" + 0.028*"miss" + 0.024*"hous" + 0.019*"search" + 0.019*"help" + 0.017*"mayor" + 0.015*"polic" + 0.014*"teen" + 0.014*"head" + 0.014*"rain"

Score: 0.22009174525737762	 
Topic: 0.022*"australia" + 0.020*"forc" + 0.018*"open" + 0.016*"iraq" + 0.016*"inquiri" + 0.015*"victim" + 0.014*"begin" + 0.013*"aust" + 0.012*"lead" + 0.012*"australian"

Score: 0.21963785588741302	 
Topic: 0.072*"polic" + 0.036*"charg" + 0.035*"kill" + 0.031*"face" + 0.023*"attack" + 0.022*"investig" + 0.018*"court" + 0.016*"probe" + 0.015*"drug" + 0.014*"arrest"

Score: 0.02001134119927883	 
Topic: 0.048*"govt" + 0.040*"urg" + 0.028*"fund" + 0.020*"boost" + 0.017*"minist" + 0.016*"health" + 0.016*"reject" + 0.015*"claim" + 0.015*"school" + 0.014*"road"

Score: 0.02000993676483631	 
Topic: 0.018*"talk" + 0.016*"deal" + 0.016*"jail" + 0.015*"trial" + 0.013*"appeal" + 0.012*"stand" + 0.011*"trade" + 0.011*"court" + 0.011*"china" + 0.011*"timor"

Score: 0.0200099367648363

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [181]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5778065919876099	 
Topic: 0.013*"water" + 0.010*"firefight" + 0.010*"bushfir" + 0.008*"blaze" + 0.007*"control" + 0.007*"condit" + 0.007*"job" + 0.007*"airport" + 0.006*"restrict" + 0.006*"upgrad"

Score: 0.2621513605117798	 
Topic: 0.043*"closer" + 0.010*"nuclear" + 0.010*"iran" + 0.009*"bird" + 0.007*"dump" + 0.006*"coal" + 0.006*"lanka" + 0.006*"bendigo" + 0.006*"riot" + 0.006*"pipelin"

Score: 0.02000618726015091	 
Topic: 0.013*"coast" + 0.011*"south" + 0.009*"north" + 0.009*"liber" + 0.008*"west" + 0.008*"flood" + 0.008*"gold" + 0.008*"poll" + 0.007*"cancer" + 0.006*"lebanon"

Score: 0.02000606246292591	 
Topic: 0.016*"govt" + 0.011*"plan" + 0.011*"fund" + 0.011*"council" + 0.010*"health" + 0.010*"urg" + 0.008*"indigen" + 0.008*"chang" + 0.008*"group" + 0.007*"union"

Score: 0.020006051287055016	 
Topic: 0.017*"miss" + 0.014*"search" + 0.010*"crew" + 0.009*"station" + 0.008*"plant" + 0.008*"grower" + 0.008*"rescu" + 0.008*"hill" + 0.008*"break" + 0.007*"light"

Score: 0.

## Step 6: Testing model on unseen document ##

In [184]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.42001181840896606	 Topic: 0.017*"world" + 0.015*"win" + 0.015*"record" + 0.014*"coast" + 0.013*"final"
Score: 0.22000038623809814	 Topic: 0.031*"water" + 0.019*"warn" + 0.018*"secur" + 0.016*"test" + 0.015*"concern"
Score: 0.21994827687740326	 Topic: 0.037*"seek" + 0.028*"miss" + 0.024*"hous" + 0.019*"search" + 0.019*"help"
Score: 0.020007099956274033	 Topic: 0.048*"govt" + 0.040*"urg" + 0.028*"fund" + 0.020*"boost" + 0.017*"minist"
Score: 0.020006360486149788	 Topic: 0.049*"plan" + 0.040*"council" + 0.017*"govt" + 0.016*"chang" + 0.015*"nation"
Score: 0.020006125792860985	 Topic: 0.022*"australia" + 0.020*"forc" + 0.018*"open" + 0.016*"iraq" + 0.016*"inquiri"
Score: 0.0200049951672554	 Topic: 0.018*"talk" + 0.016*"deal" + 0.016*"jail" + 0.015*"trial" + 0.013*"appeal"
Score: 0.0200049951672554	 Topic: 0.072*"polic" + 0.036*"charg" + 0.035*"kill" + 0.031*"face" + 0.023*"attack"
Score: 0.0200049951672554	 Topic: 0.026*"rise" + 0.023*"death" + 0.020*"case" + 0.019*"return" + 0.01

The model correctly classifies the unseen document with ~`42%` probability to the `Sports` category.